# **Colab Implementation of the Vesuvius Challenge Grand Prize Winning Solution**


 **Introduction**

This notebook is a Colab implementation of the Vesuvius Challenge Grand Prize Winning solution by Youssef Nader, Luke Farritor, and Julian Schiliger ([here](https://github.com/younader/Vesuvius-Grandprize-Winner)).

This fork of the model contains Pygosceles' significant training optimisations and speed improvements (15–20x+!) ([here]( https://github.com/StewartSethA/Vesuvius-Grandprize-Winner)).

The inference script was improved by cody256c's contributions ([here]( https://github.com/cody256c/gp-winner-modified)). It also has a fix from O_D.

The structure of this notebook was informed by Sean Johnson's (Bruniss') implementation of Youssef Nader's First Letters model ([here](https://colab.research.google.com/drive/1vvAmVas7ChFmX1xtNVYvJRtA_gsDnenv?usp=sharing)). It also contains a script from Bruniss' implementation to quickly download files to google drive.

The intention of this notebook is to make running this model more accessible, especially for those who are not very familiar with machine learning.

---

 **Training and Inference**


This solution has two processes.

The first process, [training](https://developers.google.com/machine-learning/glossary#training),  [models](https://developers.google.com/machine-learning/glossary#model) what is and isn't ink by parsing [labelled](https://developers.google.com/machine-learning/glossary#label), unrolled 'segments' of the Herculaneum papyri. The labels, 'maps' of where the ink is — or where we guess the ink is! — in the scrolls, train the model to identify ink. To produce the best results, multiple passes over the entire training set ([epochs](https://developers.google.com/machine-learning/glossary#epoch)) are performed. At the end of each epoch, the script generates a [checkpoint](https://developers.google.com/machine-learning/glossary#checkpoint). A segment is also used for [validation](https://developers.google.com/machine-learning/glossary#validation) to periodically evaluate the quality of the model.

The second process, [inference](https://developers.google.com/machine-learning/glossary#inference), uses those checkpoints to produce [predictions](https://developers.google.com/machine-learning/glossary#prediction) of where the ink is in unlabelled segments. The prediction output is an image of the scroll segment (i.e. hopefully of the text).

General introductions to machine learning can be found [here](https://developers.google.com/machine-learning/foundational-courses
).

---

 **Colab Requirements**

*   ***Google Drive Storage Space:***

    * *Training:* Space for the segments used to train the model (generally 10s of GB each) and checkpoints produced (generally hundreds of MB each).
  
     The data for the 'ink banner' segments and a single validation segment takes up around 350GB in total.      

    * *Inference:* Space for the segement to be analysed (generally 10s of GB), checkpoint used (generally hundreds of MB), and inference results (generally 10s of MB).

* ***Hardware:***

    * This notebook was tested on a Colab Pro+ subscription, utilising an A100 (40GB) with 83.5GB system RAM. It is possible to execute inference successfully (albeit with extended run times) using lower tier Colab hardware, if the number of workers etc is adjusted. For training, the 83.5 GB system RAM available is a significant bottleneck.
---


**Local Hardware**

It is possible to execute this notebook on your local hardware (Colab's instructions [here](https://research.google.com/colaboratory/local-runtimes.html)).

---

# **A. Initial Setup (Training and Inference)**

**A.1. Mount Google Drive.**

This allows files stored in google drive to be accessed by the model's scripts. (Click the square brackets below to run the command).

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**A.2.  Change directory to Google Drive.**

This ensures the Github repository and any changes you make to the scripts will be saved, even after Colab is closed.                              

In [ ]:
%cd gdrive
%cd MyDrive

/content/gdrive
/content/gdrive/MyDrive


**A.3. Clone the repository from Github.**

This creates a copy of the Github repository that can be executed by Colab.  
This command only needs to be run once. If you would like to remove any of your modifications to the scripts, delete the repository from your google drive and run this command again.

In [ ]:
!git clone https://github.com/Bodillium/Vesuvius-Grandprize-Winner

fatal: destination path 'Vesuvius-Grandprize-Winner' already exists and is not an empty directory.


**A.4. Create the directories to be used.**

This will not overwrite the directories if they already exist.

***Scroll segments not listed:*** To create a directory for a different scroll segment, simply copy+paste the relevant line below, add it to the list, and replace the segment number (e.g. 20231005123336) with the number you want before running the command. The training and/or inference scripts will need to be adjusted accordingly (instructions below).

In [ ]:
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/checkpoints' # Where to put the checkpoints used in inference
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/inference_output' # Where the inference outputs (predictions) are saved
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/volume/segments/20231005123336/layers' # The segment on which inference is to be run
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20230702185753/layers' # The segments used to train the model
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20230929220926/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231005123336/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231007101619/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231012184423/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231016151002/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231022170901/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231031143852/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231106155351/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231210121321/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20231221180251/layers'
!mkdir -p '/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/20230820203112/layers' # Current validation segment
!mkdir -p '/content/gdrive/Mydrive/vesuvius_model/training/outputs' # Where the training outputs (checkpoints, etc) are saved

mkdir: cannot create directory ‘/content/gdrive/Mydrive’: Operation not supported


**A.5. Change directory to the repository directory.**

In [ ]:
%cd Vesuvius-Grandprize-Winner

/content/gdrive/MyDrive/Vesuvius-Grandprize-Winner


**A.6. Download the layers of the segments into their respective folders.**

The 'layers' folders of each segment you intend to load must be filled with their respective .tif files from the data server.

For example, the folder /training/train_scrolls/20231005123336/layers must contain the layers 00–64.tif found [here](http://dl.ash2txt.org/full-scrolls/Scroll1.volpkg/paths/20231005123336/layers/) on the data server, to use this segment for training.

*For training:* you'll need to download the layers into the numbered layers subfolders found in training/train_scrolls for both the segments of the model you're using and the 'validation' segment.

*For inference:* you'll need to download they layers into the numbered layers subfolder of the segment you wish to use in volume/segments/.

The following script (written by Bruniss) downloads these files directly into the correct folder on your google drive (much quicker than downloading them to your computer, then uploading to google drive).

***Using the Script:***
1. Navigate to the script *downloader.py* by clicking the 'folder' icon on the left-hand side of Colab and clicking through the filepath: /content/gdrive/MyDrive/Vesuvius-Grandprize-Winner/utilities/downloader.py . Double click to open the script.
2. In **line 15** of the script, replace the word USERNAME and then the word PASSWORD with the username and password you receive after consenting to the [data agreement](https://docs.google.com/forms/d/e/1FAIpQLSf2lCOCwnO1xo0bc1QdlL0a034Uoe7zyjYBY2k33ZHslHE38Q/viewform?usp=send_form).
3. In **line 8**, change the URL to the URL you wish to download the layers from (there should be a / at the end of the URL). This needs to be repeated for each set of layers you need to download.
4. In **line 11**, change the directory to the location you wish to download the files to. The number of the file you download (e.g. 20231005123336) should match the number of the directory you are saving the files to (e.g. 20231005123336/layers).
5. Run the script below! Repeat for each set of layers you need.

In [ ]:
!python utilities/downloader.py

Download completed.


**A.7. Download the masks of the required segments into their respective folders.**

As well a a layers folder, the numbered folders of each segment you load must contain its respective 'segment mask' file from the data server. This applies to both training and inference.

For example, the folder /vesuvius_model/training/train_scrolls/20231005123336 must contain the file called 20231005123336_mask.png found [here](http://dl.ash2txt.org/full-scrolls/Scroll1.volpkg/paths/20231005123336/), to use this segment for training.

Similarly, the folder /vesuvius_model/volume/segments/20231005123336 must contain the same mask as above to run inference on this segment.

As the masks are very small, it  may be quicker to download them from the file server and upload them to your google drive directly, instead of using the previous script.


**A.8. Install the required dependencies.**

This installs the software the scripts depend on to function.

In [ ]:
!pwd
!pip install -r requirements.txt

/content/gdrive/MyDrive/Vesuvius-Grandprize-Winner
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB

# **B. Training Only**

**B.1. Select your training set**  
If you are running training on the segments listed originally in A.4., nothing needs to be adjusted.

***Scroll segments not listed:*** Otherwise, you will need to adjust **line 363** in *64x64_256stride_i3d.py* to match your training set. Add the numbers of the segments you need to the lists of numbers in this line, and remove the numbers you are not using. Adjust **line 100** to match your validation segment, if you have changed it from the default (20230820203112).

**B.2. Manually upload ink labels**
  
Upload the ink labels you intend to use into the folder with the following path: /content/gdrive/MyDrive/Vesuvius-Grandprize-Winner/all_labels/. It is easiest to do this directly in your google drive.

Labels are included with the GP Winning model. Some updated labels can be found via the Discord [here](https://discord.com/channels/1079907749569237093/1208440488437350500).

**B.3. Propagate the ink labels into their respective folders**    

Run the following script to place the ink labels into their correct folders on your google drive.

***Scroll segments not listed:*** If you intend to run training on segments not listed in section A.4. above, you must adjust the script before propagating the ink labels. Add the segment numbers (which correspond to the directory folder names) to the lists in lines **27** and **41** of *prepare.py*.


*Note*: If you are using the ink label provided with the Grand Prize winning model for segment 20231022170901, remove the # and space in front of lines **46, 47, 48**. It has special instructions as the file is .tif, not .png. You need not do this if the ink label for this segment is a .png from another source.


In [ ]:
!python prepare.py

20230702185753
Traceback (most recent call last):
  File "/content/gdrive/MyDrive/Vesuvius-Grandprize-Winner/prepare.py", line 63, in <module>
    run_sanity_checks()
  File "/content/gdrive/MyDrive/Vesuvius-Grandprize-Winner/prepare.py", line 43, in run_sanity_checks
    assert os.path.exists(f'/content/gdrive/MyDrive/vesuvius_model/training/train_scrolls/{fragment_id}/layers/00.tif')
AssertionError


**B.4. Run Training**
    
Press the button and good luck!


***wandb question:*** A wandb (Weights & Biases, an external data tracking and visualisation tool) question will appear after the data is loaded. Type 3 to continue to training without utilising this service.

Your checkpoints after each epoch will be saved to the following directory: '/content/gdrive/mydrive/vesuvius_model/training/outputs. Training may take a *long* time.
    
***Potential Issues***
 *  *Not Enough RAM*: The RAM requirements for training are very large. If a ^C error appears while running the script, you'll need to reduce the size of the dataset.

 *  *Colab Timeouts*: Colab times out after 1.5 hours of inactivity, interrupting the script. Training will almost certainly take longer than this. You'll need to either interact with Colab every 1.5 hours to avoid this, or use another solution to click the screen automatically.

*Note:* The script is currently set up to run on an A100 (40GB) with 83.5GB system RAM. If you are using different hardware you may need to adjust the number of workers (num_workers) and other settings in the script.

In [ ]:
!python 64x64_256stride_i3d.py

set dataset path
reading  20231005123336
537.4818818569183 seconds taken to load images.
^C


# **C. Inference Only**

**C.1. Select the segment to run inference on**

If you are running training on the segment listed originally in A.4. (20231005123336), nothing needs to be adjusted.

***Scroll segment not listed:*** Otherwise, you will need to adjust **line 28** of *inference_timesformer.py* to match the segment you selected.

**C.2. Upload the checkpoint**

Upload the checkpoint you wish to use to the /content/gdrive/MyDrive/vesuvius_model/checkpoints folder. The checkpoints used in the Grand Prize winning solution can be found [here](https://drive.google.com/drive/folders/1rn3GMOvtJRMBHOxVhWFVSY6IVI6xUnYp).

**C.3. Adjust the script to match the checkpoint**

Adjust **line 30** of *inference_timesformer.py* to match the checkpoint name. The default name is the checkpoint used in the Grand Prize winning solution.

***C.4 Free Colab Only:***

*T4 GPU*: Adjust **line 77** of *inference_timesformer.py* to: num_workers = **2**

**C.5. Run Inference**
    
Press the button and good luck!

The results (predictions/images) will be saved in the /vesuvius_model/inference_output folder.  



In [ ]:
!python inference_timesformer.py

# **D. Contact**

If you have any suggestions for improving this Colab or need help, please get in touch with polytrope. on Discord!